# 一切皆是对象

In [2]:
def hi(name="yasoob"):
    return "hi " + name

print(hi())
# output: 'hi yasoob'

hi yasoob


In [4]:
# 我们甚至可以将一个函数赋值给一个变量，比如
greet = hi
print(greet())

hi yasoob


In [5]:
# 如果我们删掉旧的hi函数，看看会发生什么！
del hi
print(hi())

NameError: name 'hi' is not defined

In [6]:
print(greet())

hi yasoob


# 在函数中定义函数

In [10]:
def hi(name="yasoob"):
    print("now you are inside hi() function")
    
    def greet():
        return "now you are inside greet() function"
    
    def welcome():
        return "now you are inside welcome() function"
    print(greet())
    print(welcome())
    print("now you are back in the hi() function")
hi()

now you are inside hi() function
now you are inside greet() function
now you are inside welcome() function
now you are back in the hi() function


# 从函数中返回函数

In [11]:
def hi(name="yasoob"):
    def greet():
        return "now you are in the greet() function"
 
    def welcome():
        return "now you are in the welcome() function"
 
    if name == "yasoob":
        return greet
    else:
        return welcome
    
a = hi()
print(a)

print(a())

<function hi.<locals>.greet at 0x10be63830>
now you are in the greet() function


# 将函数作为参数传给另一个函数

In [13]:
def hi():
    return "hi yasoob!"

def doSomethingBeforeHi(func):
    print("I am doing some boring work before executing hi()")
    print(func())
    
doSomethingBeforeHi(hi)

I am doing some boring work before executing hi()
hi yasoob!


# 第一个装饰器

In [17]:
def a_new_decorator(a_func):
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
    return wrapTheFunction

def a_function_requiring_decoration():
    print("I am the function which needs some decoration to remove my foul smell")
    
# a_function_requiring_decoration()

# 伪装饰器
a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)
a_function_requiring_decoration()

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()


In [22]:
@a_new_decorator
def a_function_requiring_decoration():
    print("I am the function which needs some decoration to "
          "remove my foul smell")
    
a_function_requiring_decoration()

print("-" * 80)

#the @a_new_decorator is just a short way of saying:
a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)
a_function_requiring_decoration.__name__

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()
--------------------------------------------------------------------------------


'wrapTheFunction'

In [23]:
"""
这并不是我们想要的！Ouput输出应该是"a_function_requiring_decoration"。这里的函数被warpTheFunction替代了。它重写了我们函数的名字和注释文档(docstring)。幸运的是Python提供给我们一个简单的函数来解决这个问题，那就是functools.wraps。
我们修改上一个例子来使用functools.wraps：
"""
from functools import wraps

def a_new_decorator(a_func):
    @wraps(a_func)
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
    return wrapTheFunction

@a_new_decorator
def a_function_requiring_decoration():
    """Hey yo! Decorate me!"""
    print("I am the function which needs some decoration to "
          "remove my foul smell")
    
print(a_function_requiring_decoration.__name__)

a_function_requiring_decoration


# 现在好多了。我们接下来学习装饰器的一些常用场景。蓝本规范:

In [26]:
from functools import wraps

def decorator_name(f):
    @wraps(f)
    def decorated(*args,**kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args,**kwargs)
    return decorated

@decorator_name
def func():
    return "Function is running"

can_run = True
print(func())

can_run = False
print(func())

Function is running
Function will not run


# 授权(Authorization)

In [ ]:
"""
装饰器能有助于检查某个人是否被授权去使用一个web应用的端点(endpoint)。
它们被大量使用于Flask和Django web框架中。这里是一个例子来使用基于装饰器的授权：
"""

from functools import wraps
 
def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        auth = request.authorization
        if not auth or not check_auth(auth.username, auth.password):
            authenticate()
        return f(*args, **kwargs)
    return decorated

# 日志(Logging)

In [31]:
from functools import wraps
 
def logit(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + " was called")
        return func(*args, **kwargs)
    return with_logging

@logit
def addition_func(x):
   """Do some math."""
   return x + x

result = addition_func(4)
result

addition_func was called


8

# 带参数的装饰器

In [33]:
from functools import wraps

def logit(logfile="out.log"):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args,**kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile并且写入内容
            with open(logfile,'a') as opened_file:
                # 现在将日志打到指定的logfile
                opened_file.write(log_string + "\n")
            return func(*args,**kwargs)
        return wrapped_function
    return logging_decorator

@logit()
def myfunc1():
    pass

myfunc1()

myfunc1 was called


In [34]:
@logit(logfile="func2.log")
def myfunc2():
    pass

myfunc2()

myfunc2 was called


# 装饰器类

In [36]:
from functools import wraps

class logit(object):
    def __init__(self,logfile="out.log"):
        self.logfile = logfile
        
    def __call__(self, func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile并写入
            with open(self.logfile, 'a') as opened_file:
                # 现在将日志打到指定的文件
                opened_file.write(log_string + '\n')
            # 现在，发送一个通知
            self.notify()
            return func(*args, **kwargs)
        return wrapped_function
 
    def notify(self):
        # logit只打日志，不做别的
        pass

@logit()
def myfunc1():
    pass

myfunc1()

myfunc1 was called
